In [1]:
# Kaggle cell: resume training + push checkpoints to HF
!pip -q install -U transformers datasets accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 90.7 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 M

In [1]:
import os, math
from pathlib import Path
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login, snapshot_download
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM,
    DataCollatorForLanguageModeling, Trainer, TrainingArguments, Early
)

2025-08-12 12:49:30.636261: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755002970.879564      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755002970.948294      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
hf_token = UserSecretsClient().get_secret("HF_TOKEN")
os.environ["HUGGING_FACE_HUB_TOKEN"] = hf_token
login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
repo = "mhurhangee/patroberta-mlm-sl128-v8000"

In [4]:
cache_root = snapshot_download(repo_id=repo, repo_type="model")
resume_ckpt = cache_root  # points to the folder with optimizer.pt, trainer_state.json, etc.


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/6.05M [00:00<?, ?B/s]

...

scaler.pt:   0%|          | 0.00/988 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

trainer_state.json: 0.00B [00:00, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

In [42]:
tokenizer = AutoTokenizer.from_pretrained(repo, use_fast=True)
model = AutoModelForMaskedLM.from_pretrained(resume_ckpt or repo)

In [6]:
ds = load_dataset("mhurhangee/ep-patent-all-claims")
train_ds, val_ds = ds["train"], ds["validation"]

README.md:   0%|          | 0.00/385 [00:00<?, ?B/s]

data/train-00000-of-00006.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

data/train-00001-of-00006.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

data/train-00002-of-00006.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

data/train-00003-of-00006.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

...

data/validation-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4243904 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/88415 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/88415 [00:00<?, ? examples/s]

In [7]:
seq_len = 128
def enc(b): return tokenizer(b["text"], truncation=True, max_length=seq_len)
train_ds = train_ds.map(enc, batched=True, remove_columns=train_ds.column_names)
val_ds   = val_ds.map(enc,   batched=True, remove_columns=val_ds.column_names)

Map:   0%|          | 0/4243904 [00:00<?, ? examples/s]

Map:   0%|          | 0/88415 [00:00<?, ? examples/s]

In [43]:
print(train_ds.column_names)   # should be ['input_ids','attention_mask'] (maybe 'token_type_ids')
print(val_ds.column_names)
print(train_ds[0].keys())      # verify same

['input_ids', 'attention_mask']
['input_ids', 'attention_mask']
dict_keys(['input_ids', 'attention_mask'])


In [44]:
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [61]:
SEQ_LEN = 128                 
ARCH_MAX_POSITIONS = 514       # capacity (OK for 512 later)
MLM_PROB = 0.15

# Tiny model
HIDDEN_SIZE, NUM_LAYERS, NUM_HEADS, INTER_SIZE = 128, 2, 2, 512
DROPOUT = 0.1

# Training knobs
PER_DEVICE_TRAIN_BS = 512
PER_DEVICE_EVAL_BS  = 64
GRAD_ACCUM_STEPS    = 1        # effective batch = 32 sequences
LEARNING_RATE       = 5e-4
WEIGHT_DECAY        = 0.01
WARMUP_RATIO        = 0.06
FP16                = True
NUM_EPOCHS          = 24

# Early stopping
PATIENCE = 3                     # evals with no improvement
THRESH   = 1e-4                  # min improvement in eval_loss

In [62]:
model.gradient_checkpointing_disable()

In [63]:
args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=PER_DEVICE_TRAIN_BS,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BS,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    num_train_epochs=12,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=4,
    logging_steps=1657,
    fp16=FP16,
    fp16_full_eval=True,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    remove_unused_columns=False,
    # Hub pushing
    push_to_hub=True,
    hub_model_id=repo,
    hub_token=hf_token,
    hub_private_repo=False,
    hub_strategy="every_save",   # push each checkpoint on save
)

In [64]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    data_collator=collator,

)

In [37]:
# 1) Hard-clear GPU without touching your datasets
import os, gc, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
for name in ("trainer","model"):
    if name in globals(): del globals()[name]
gc.collect(); torch.cuda.empty_cache(); 
if hasattr(torch.cuda, "ipc_collect"): torch.cuda.ipc_collect()

In [80]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
3315,2.229400,1.956403
6630,2.173700,1.902118
9945,2.134500,1.867756
13260,2.110400,1.843583
16575,2.091000,1.826831
19890,2.077900,1.812651
23205,2.064700,1.803954
26520,2.055800,1.790961
29835,2.049400,1.786600
33150,2.043800,1.779748


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=33160, training_loss=2.108681117035943, metrics={'train_runtime': 23056.2627, 'train_samples_per_second': 1472.538, 'train_steps_per_second': 1.438, 'total_flos': 1.0992194282323968e+16, 'train_loss': 2.108681117035943, 'epoch': 8.0})

In [ ]:
out = trainer.evaluate()
import math as _m
print(out, "Perplexity:", _m.exp(out["eval_loss"]))

In [82]:
trainer.save_model()                 # saves config.json + model.safetensors
tokenizer.save_pretrained(args.output_dir)
trainer.push_to_hub()                # one final commit

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...rking/checkpoints/training_args.bin: 100%|##########| 5.37kB / 5.37kB            

  ...rking/checkpoints/model.safetensors: 100%|##########| 6.05MB / 6.05MB            

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...rking/checkpoints/training_args.bin: 100%|##########| 5.37kB / 5.37kB            

  ...rking/checkpoints/model.safetensors: 100%|##########| 6.05MB / 6.05MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mhurhangee/patroberta-mlm-sl128-v8000/commit/a3b25838b8711c13fab842e146232dc69c2a0e1f', commit_message='End of training', commit_description='', oid='a3b25838b8711c13fab842e146232dc69c2a0e1f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mhurhangee/patroberta-mlm-sl128-v8000', endpoint='https://huggingface.co', repo_type='model', repo_id='mhurhangee/patroberta-mlm-sl128-v8000'), pr_revision=None, pr_num=None)

In [ ]:
trainer.train(resume_from_checkpoint=resume_ckpt)


You are resuming training from a checkpoint trained with 4.55.0 of Transformers but your current version is 4.52.4. This is not recommended and could yield to errors or unwanted behaviors.
There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].
	save_steps: 500 (from args) != 3315 (from trainer_state.json)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [83]:
trainer.train(resume_from_checkpoint=resume_ckpt)


-rw-r--r-- 1 root root  12M Aug 12 10:42 checkpoints/checkpoint-26520/optimizer.pt
-rw-r--r-- 1 root root 1.1K Aug 12 10:42 checkpoints/checkpoint-26520/scheduler.pt
-rw-r--r-- 1 root root  12M Aug 12 11:20 checkpoints/checkpoint-29835/optimizer.pt
-rw-r--r-- 1 root root 1.1K Aug 12 11:20 checkpoints/checkpoint-29835/scheduler.pt
-rw-r--r-- 1 root root  12M Aug 12 11:59 checkpoints/checkpoint-33150/optimizer.pt
-rw-r--r-- 1 root root 1.1K Aug 12 11:59 checkpoints/checkpoint-33150/scheduler.pt
-rw-r--r-- 1 root root  12M Aug 12 11:59 checkpoints/checkpoint-33160/optimizer.pt
-rw-r--r-- 1 root root 1.1K Aug 12 11:59 checkpoints/checkpoint-33160/scheduler.pt


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
